# **Импорт и предобработка**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers 

In [3]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn import svm
import lightgbm as lgb
from xgboost import XGBClassifier
import torch
import transformers 
from sklearn import model_selection, naive_bayes
import numpy as np
from transformers import BertModel, BertConfig, BertTokenizer
from tqdm import notebook
from nltk.stem import WordNetLemmatizer 
from sklearn.metrics import precision_score
from sklearn.neural_network import MLPClassifier
from sklearn import svm

In [4]:
df=pd.read_csv('/content/drive/MyDrive/NLP test/labeled.csv')
df['toxic']=df['toxic'].astype('int')

In [5]:
df['toxic'].value_counts()

0    9586
1    4826
Name: toxic, dtype: int64

In [6]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
all_lemm_sentences = []
lemmatizer = WordNetLemmatizer()
for i in range(df.shape[0]):
    sentence = df.loc[i]['comment']
    word_list = re.sub(r'[^a-zA-Zа-яА-ЯёЁ ]', ' ', df.loc[i]['comment']).split()
    lemmatized_output =  ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    lemmatized_output=str(lemmatized_output).lower()
    all_lemm_sentences.append(lemmatized_output)

In [8]:
df['clear']=all_lemm_sentences

In [9]:
df['clear'][1]

'хохлы это отдушина затюканого россиянина мол вон а у хохлов еще хуже если бы хохлов не было кисель их бы придумал'

In [10]:
train, test = train_test_split(df, test_size=0.25, random_state=123, stratify=df['toxic'])
print(train['toxic'].value_counts())
print(test['toxic'].value_counts())

0    7189
1    3620
Name: toxic, dtype: int64
0    2397
1    1206
Name: toxic, dtype: int64


In [11]:
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
train_target=train['toxic']
train_corpus=train['clear']
count_tf_idf = TfidfVectorizer(stop_words=stop_words)
train_features = count_tf_idf.fit_transform(train_corpus) 
test_target=test['toxic']
test_corpus=test['clear']
test_features = count_tf_idf.transform(test_corpus) 

In [13]:
train_features.shape

(10809, 55606)

In [14]:
train_target.shape

(10809,)

In [15]:
#чтоб не ебаться с обучением и скорами
def fit_predict(model):
    model.fit(train_features, train_target)
    predict=model.predict(test_features)
    f1=f1_score(test_target, predict)
    precision=precision_score(test_target, predict)
    accuracy=accuracy_score(test_target, predict)
    recall=recall_score(test_target, predict)
    print("Model recall:", recall)
    print("Model accuracy:", accuracy)
    print("Model precision:", precision)
    print("F1 модели на тестовой выборке:", f1)

# **Обучение моделей**

In [16]:
model=LogisticRegression(max_iter=10000, random_state=123)
fit_predict(model)

Model recall: 0.45439469320066334
Model accuracy: 0.8037746322509021
Model precision: 0.9179229480737019
F1 модели на тестовой выборке: 0.6078757626178591


In [17]:
model_lgb=lgb.LGBMClassifier(boosting_type='gbdt',  max_depth=10, 
                             learning_rate=0.0001, n_estimators=5000, class_weight='balanced')
fit_predict(model_lgb)

Model recall: 0.8441127694859039
Model accuracy: 0.5384401887316126
Model precision: 0.4083433614119535
F1 модели на тестовой выборке: 0.550419032170857


In [18]:
model_xg=XGBClassifier(eta=0.1, max_depth= 10,
                       sampling_method = 'gradient_based', class_weight='balanced')
fit_predict(model_xg)

Model recall: 0.2860696517412935
Model accuracy: 0.7474326949764085
Model precision: 0.8756345177664975
F1 модели на тестовой выборке: 0.43124999999999997


In [19]:
clf = MLPClassifier(solver='sgd', alpha=1e-4, hidden_layer_sizes=(16, 1), random_state=123,
                     warm_start=True, max_iter=10000)
fit_predict(clf)

Model recall: 0.0
Model accuracy: 0.6652789342214821
Model precision: 0.0
F1 модели на тестовой выборке: 0.0


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
svm= svm.SVC(kernel='linear', gamma="auto")
fit_predict(svm)

Model recall: 0.6600331674958541
Model accuracy: 0.8498473494310297
Model precision: 0.8586839266450917
F1 модели на тестовой выборке: 0.7463666197843413


# **Сезам**

In [21]:
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
config = transformers.DistilBertConfig.from_pretrained('distilbert-base-uncased')
model = transformers.DistilBertModel.from_pretrained('distilbert-base-uncased', config=config)

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you 

In [22]:
def features_embeddings(features, batch_size):

  tokenized = features.apply((lambda x: tokenizer.encode(x, max_length=512, add_special_tokens=True, truncation=True)))
  max_len = 0
  for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i) 
  padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
  attention_mask = np.where(padded != 0, 1, 0)

  embeddings = []
  for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
    with torch.no_grad():
      batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
    embeddings.append(batch_embeddings[0][:,0,:].numpy())



  features_from_embeddings = np.concatenate(embeddings)

  return features_from_embeddings

In [23]:
features = df['comment'][:1000]
target = df['toxic'][:1000]
features_embeddings = features_embeddings(features, 100)
features_embeddings = pd.DataFrame(features_embeddings)

  0%|          | 0/10 [00:00<?, ?it/s]

In [24]:
features=features_embeddings

In [25]:
train_features, test_features, train_target, test_target = train_test_split(features, target, test_size=0.33, random_state=1234, stratify=target)

In [26]:
model=LogisticRegression(max_iter=10000, random_state=123)
fit_predict(model)


Model recall: 0.8341708542713567
Model accuracy: 0.6393939393939394
Model precision: 0.6587301587301587
F1 модели на тестовой выборке: 0.7361419068736141


In [27]:
model_lgb=lgb.LGBMClassifier(boosting_type='gbdt',  max_depth=10, 
                             learning_rate=0.005, n_estimators=5000, loss='log')
fit_predict(model_lgb)

Model recall: 0.8190954773869347
Model accuracy: 0.6333333333333333
Model precision: 0.657258064516129
F1 модели на тестовой выборке: 0.7293064876957494


In [28]:
model_xg=XGBClassifier(eta=0.1, max_depth= 10,
                       sampling_method = 'gradient_based', class_weight='balanced')
fit_predict(model_xg)

Model recall: 0.8040201005025126
Model accuracy: 0.6303030303030303
Model precision: 0.6584362139917695
F1 модели на тестовой выборке: 0.7239819004524888


In [29]:
clf = MLPClassifier(solver='sgd', alpha=1e-4, hidden_layer_sizes=(16, 1), random_state=123,
                     warm_start=True, max_iter=100000)
fit_predict(clf)

Model recall: 0.8693467336683417
Model accuracy: 0.6545454545454545
Model precision: 0.6628352490421456
F1 модели на тестовой выборке: 0.7521739130434782


In [33]:
# for kernel in ('linear','poly', 'rbf'):
#     classifier_svm = svm.SVC(kernel=kernel, C=1)
#     classifier_svm.fit(train_features, train_target)
#     predict=classifier_svm.predict(test_features)
#     print("testing success ratio with "+ kernel + "kernel :" + str(accuracy_score(test_target, predict)))


AttributeError: ignored